In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import glob
import pickle

from music21 import converter, instrument, stream, note, chord
import tensorflow as tf
import music21 as m21


from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import TimeDistributed

In [83]:
# This is the architecture of the network with the lowest error 
# Notice the attention width is the same as input sequence
def build_model(lstm_input):
    '''
    Build and compile the model
    
    lstm_input: lstm_input.shape[1] = number of steps, lstm_input.shape[2] = number features needed for
    Bi-directional
    
    model_output: number of categories to output for classification
    
    returns: compiled model
    '''
    
    model = Sequential()
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    model.add(SeqSelfAttention(attention_width = 100 , attention_activation='sigmoid'))
    model.add(BatchNorm())
    model.add(Dropout(0.2))
    
    
    
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(512, return_sequences=False)))
    
    
    model.add(Flatten())
    model.add(Dense(VOCAB))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    #model.load_weights('models_weights/weights_attention.h5')
    return model
# Load the model first -- >tf.keras.models.load_model((path),custom_objects={'SeqSelfAttention': SeqSelfAttention})

In [13]:
def prepare_notes(notes, pitches, VOCAB):
    '''
    prepare lstm input notes again used by the network to predict notes
    
    '''
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_normalized = lstm_input / float(len(pitches))
    
    return (lstm_input, lstm_normalized)
lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB)

In [24]:
def predict_to_notes(model, lstm_input, pitches, VOCAB):
    '''
    **Generated predictions from model based on random starting point**
    
    model: --> Original model with weights loaded
    lstm_inputL: --> Output from prepare, used to initialize pattern with an int the model recognizes
    pitches: ---> list of all the notes, chords and rests
    VOCAB:----> Number of unique notes to classify = len(pitches)
    
    output: Predicted_notes 
    '''
    #random starting point 
    start = np.random.randint(0, len(lstm_input) -1)
    note_dict = dict((number, note) for number, note in enumerate(pitches))
    
    pattern = lstm_input[start]
    print(type(pattern))
   
    predicted_notes = []
    
    
    for note in range(500):
        to_predict = np.reshape(pattern, (1, len(pattern), 1))
        to_predict = to_predict/ float(VOCAB)
        #print(to_predict.shape)
        
        prediction = model.predict(to_predict, verbose = 0)
        #print(type(prediction))
        index = np.argmax(prediction)
        print(index)
        
        result = note_dict[index]
        predicted_notes.append(result)
        
        pattern = np.append(pattern, index)
        print(pattern)
        pattern = pattern[1:len(pattern)]
        
    return predicted_notes

In [20]:
def midi_convert(predicted_notes):
    '''
    convert the notes in predicted_notes to midi files
    
    predicted_notes: Output from function predict_to_notes() --> list of predicted notes
    
    returns: None --- > Creates a midi file when ran
    '''
    offset = 0 
    midi_notes = []
    
    #create notes, chords, and rest objects from predicted_notes
    for pattern in predicted_notes:
        pattern = pattern.split(' ')
        print(pattern)
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        #checking to see if a note is a chord 
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = []
            for current_note in chord_notes:
                this_note = note.Note(int(current_note))
                this_note.storedInstrument = instrument.Piano()
                notes.append(this_note)
            new_chord = chord.Chord(notes) 
            new_chord.offset = offset
            midi_notes.append(new_chord)
        #if the pattern is a rest    
        elif ('rest' in pattern):
            this_rest = note.Rest(pattern)
            this_rest.offset = offset
            this_rest.storedInstrument = instrument.Piano() #Still needs to be paino instrument even though = rest
            midi_notes.append(this_rest)
        else:
            this_note = note.Note(pattern)
            this_note.offset = offset 
            this_note.storedInstrument = instrument.Piano()
            midi_notes.append(this_note)
        #ensure that the notes do not stack    
        offset += convert_to_float(duration)
    
    midi = stream.Stream(midi_notes)
    midi.write('midi', fp = 'midi_output/new_try.mid')
    
    
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [23]:
def generate():
    '''
    generates the midi file
    
    
    
    output:None ---> creates the midi file
    
    '''
    #Load the notes
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    
    #create the pitchnames
    pitches = sorted(set(note for note in notes))
    
    
    lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB) 
    model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
    model.load_weights('models_weights/weights_no_attentnion.hdf5)
    predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)
    midi_convert(predicted_notes)
    

In [16]:
model.load_weights('models_weights/weights_no_attentnion.hdf5')
#model = tf.keras.models.load_model('models_weights/attention_model.h5')

In [9]:
#model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
path = 'models_weights/Attention_model_legit.h5'

In [10]:
model = tf.keras.models.load_model((path),custom_objects={'SeqSelfAttention': SeqSelfAttention})
model.load_weights('models_weights/Attention_weights.h5')

In [22]:
#midi_convert(predicted_notes)

In [ ]:
## TESTING ##
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)

#using these just to test
VOCAB = (len(set(notes)))
pitches = sorted(set(note for note in notes))